# Example of using SpecialtyInsurance Simulator


In [1]:
### Set Simulation Parameters
import os
from logger.arguments import get_arguments

# Read arguments from logger.arguments
sim_args, manager_args, broker_args, syndicate_args, reinsurancefirm_args, shareholder_args, risk_args = get_arguments()

# Reset arguments
sim_args["max_time"] = 300   # Simulation time span unit day
manager_args["lead_top_k"] = 2   # Number of syndicates competing for the lead quote
manager_args["follow_top_k"] = 1   # Number of syndicates following the lead strategy
broker_args["num_brokers"] = 1   # Number of brokers in the insurance market
syndicate_args["num_syndicates"] = 3   # Number of syndicates in the insurance market
syndicate_args["lead_line_size"] = 0.90  # The percentage of risk covered by lead syndicate
syndicate_args["follow_line_size"] = 0.10  # The percentage of risk covered by follow syndicate
shareholder_args["num_shareholders"] = 1   # Number of shareholders in the insurance market
risk_args["num_risks"] = 1  # Number of risk categories
risk_args["num_categories"] = 4  # Number of risk categories

# No reinsurance mechanism included in this stage
with_reinsurance = False   

# Load one risk model to all syndicates
num_risk_models = 1   

In [2]:
# Generate Catastrophes
from environment.risk_generator import RiskGenerator

# Create catastrophe list and catastrophe configurations
catastrophes, risk_model_configs = RiskGenerator(num_risk_models, sim_args, risk_args).generate_risks()
print(catastrophes)

[{'risk_id': 0, 'risk_start_time': 169, 'risk_factor': 0.5141633664737083, 'risk_category': 0, 'risk_value': 1000.0}]


In [3]:
# Generate Insurance Market
from environment.market_generator import MarketGenerator

# Create lists of brokers, syndicates, reinsurancefirms, and shareholders
brokers, syndicates, reinsurancefirms, shareholders = MarketGenerator(with_reinsurance, 
                                                                      num_risk_models, 
                                                                      sim_args, 
                                                                      broker_args, 
                                                                      syndicate_args, 
                                                                      reinsurancefirm_args, 
                                                                      shareholder_args, 
                                                                      risk_model_configs).generate_agents()

In [4]:
# Input risk from broker

# Risk brought by broker: time to the market and number of risks
time = 0
num_risk = 1

for i in range(len(brokers)):
    risks = brokers[i].generate_risks(time, num_risk)
    print(risks)

[{'risk_id': 0, 'broker_id': 0, 'risk_start_time': 0, 'risk_end_time': 360, 'risk_factor': 0.5, 'risk_category': 4, 'risk_value': 1000.0}]


In [5]:
# Run the simulation
from manager.ai_model.runner import AIRunner
from manager.game_model.runner import GameRunner

model = 0
if model == 0: 
    runner = AIRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
elif model == 1:
    runner = GameRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
runner.run()

/opt/homebrew/lib/python3.8/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/homebrew/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/opt/homebrew/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could 

TypeError: run() got an unexpected keyword argument 'step_time'